# Table of Contents
 <p>

In [12]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from mpl_toolkits.basemap import Basemap
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from netCDF4 import Dataset, MFDataset
from math import *
import glob
import string
from datetime import date, time, datetime
from pprint import pprint
import pandas as pd
import sys,os
import StringIO

# Widen displayed cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# Add ../functions to system path
currentpath = %pwd
currentpath = str(currentpath)
##-- In a python script do the following instead:
##-- currentpath = os.path.dirname(os.path.realpath(__file__))
sys.path.insert(0,currentpath[:currentpath.rfind('/')+1]+'functions')

from plotting import *
from thermodynamics import *

### Figure setup
# Set up fontsizes
plt.rcParams.update({'axes.labelsize': 'x-large',
                     'axes.titlesize': 'x-large',
                     'xtick.labelsize': 'large',
                     'ytick.labelsize': 'large',
                     'figure.titlesize': 'xx-large',
                     'figure.subplot.top': '0.87',
                     'figure.subplot.wspace': '0.2',
                     'figure.subplot.hspace': '0.3'})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Load an SPCAM output file
dir_spcam = "/Users/bfildier/Data/spcam/sample_outputs/"
file_spcam = "base_variables.cam.h0.2008-10-15-03600.nc"
inputfile_spcam = os.path.join(dir_spcam,file_spcam)
fh_spcam = Dataset(inputfile_spcam,mode = 'r')
nvar = len(fh_spcam.variables.keys())
var_info = pd.DataFrame(index=range(0),columns=['Variable','Size','Dimensions','Name'])
for i in range(nvar):
    var = fh_spcam.variables.keys()[i]
    # Dimensions names
    desc_stream = StringIO.StringIO()
    print >>desc_stream, fh_spcam.variables[var]
    dimnames = desc_stream.getvalue().split(' '+var)[1].split('\n')[0]
    # Var name
    name = ''
    if hasattr(fh_spcam.variables[var], 'long_name'):
        name = fh_spcam.variables[var].long_name
    # Store output
    var_info.loc[i] = [var,fh_spcam.variables[var].shape,dimnames,name]
var_info.set_index("Variable",inplace=True)
var_info.index.names = [None]

print "\tVariables contained in IASI files:\n"
print var_info.to_string(justify="right")

	Variables contained in IASI files:

                                Size                                Dimensions                                               Name
lev                            (30,)                                     (lev)             hybrid level at midpoints (1000*(A+B))
hyam                           (30,)                                     (lev)            hybrid A coefficient at layer midpoints
hybm                           (30,)                                     (lev)            hybrid B coefficient at layer midpoints
ilev                           (31,)                                    (ilev)            hybrid level at interfaces (1000*(A+B))
hyai                           (31,)                                    (ilev)           hybrid A coefficient at layer interfaces
hybi                           (31,)                                    (ilev)           hybrid B coefficient at layer interfaces
P0                                ()                 

In [8]:
Q = fh_spcam.variables['Q'][:]
T = fh_spcam.variables['T'][:]
RELHUM = fh_spcam.variables['RELHUM'][:]
RH = relativeHumidity(T,Q)
print Q.shape, T.shape, RELHUM.shape
print Q.min(), Q.max()
print T.min(), T.max()
print np.mean(RH[0,-1,:,:]), RH[0,-1,:,:].std(), RELHUM[0,0,:,:].mean(), RELHUM[0,0,:,:].std()

(1, 30, 96, 144) (1, 30, 96, 144) (1, 30, 96, 144)
9.50312e-07 0.0211684
186.605 310.564
0.000776513482332 0.000173158153188 0.025937 0.0600587


In [45]:
import StringIO

var = 'V'
desc_stream = StringIO.StringIO()
print >>desc_stream, fh_spcam.variables[var]
dimnames = desc_stream.getvalue().split(' '+var)[1].split('\n')[0]
print dimnames
# print fh_spcam.dimensions.keys()
# for i in fh_spcam.variables['CRM_T'].mdims:
#     print fh_spcam.dimensions.keys()[i]

(time, lev, lat, lon)
